In [ ]:
## This file implements neural networks and logistic regression on p0017presabs_quant.
## For fully-connected neural networks, the accuracy is 64.94%, and 66.23% after improvememt.
## To improve accuracy, we construct lasso regression to select important features, and thus form a new dataset.
## With the new dataset, the accuracy is 62.34%, and keeps 66.23% after improvement.
## For logistic regression, the accuracy is 62.34%.
## We further construct random forest models, which are relatively less likely to bring overfitting 
## compared to decision tree.
## For random forest, the accuracy is 63.64%.
## For random forest with cross-validation, the mean accuracy is 75.26%.

In [219]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [161]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p0017presabs_quant.csv')
df.shape

(255, 7004)

In [162]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [163]:
df['pheno']

0      0.871375
1      0.364375
2      0.803000
3      0.952625
4      1.012875
5      0.936750
6      0.745375
7      0.348375
8      0.571750
9      0.637125
10     0.670500
11     0.731250
12     0.865125
13     0.569375
14     0.327000
15     0.175750
16     0.964000
17     1.104167
18     0.223667
19     0.340167
20     0.438500
21     0.271833
22     0.248833
23     0.739333
24     0.232833
25     0.601500
26     0.251000
27     0.202833
28     0.599833
29     0.260667
         ...   
225    0.459500
226    0.825333
227    0.763167
228    0.334500
229    0.998833
230    0.491125
231    0.405750
232    0.875375
233    0.761500
234    0.255000
235    0.455125
236    0.234125
237    0.340125
238    0.952750
239    0.238500
240    0.294500
241    0.172125
242    0.221500
243    0.989875
244    0.763000
245    0.229875
246    0.215125
247    0.805375
248    0.184875
249    0.663250
250    0.803125
251    0.443875
252    0.207500
253    0.933875
254    0.873375
Name: pheno, Length: 255

In [164]:
df.head()

,id,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTGTAAATCTTTTCCATTTATTTCTTCGCTTTTCTCGCTACAATTTACAAATGGACTTGTTATCAACGTTAGGGCAATAAATGAAAGTAGTATAA,TTTTTTTGATAATTT,TTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAAAAAACGTATTTAATGTTGACAATACGAAAATTTCGTATTATATTAGGT,TTTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCA,TTTTTTTATTTTGCAATTTTTTATTTTCATTATAAACTTCCTTTCAAACACTGCTGAAATAGACGTCTTTTTCAAATAAGCATGATTAATACTTCAATTC,TTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAGT,TTTTTTTATGTTATTATAATTAAAGTTTTCCATTGTTTTCCTCCTATAATAGCTTATCTGCAATCATCACAGCTAATAAATCGTTTTGTCTTATTGCTTC,TTTTTTTATGTTATAATCTTTCTAGACGTATTCAAAGGACGTCTTTTTAGATTGTATGTTATAGCTAGCCTTCCGGTTAATTTTTTGTTATGATGTGTTA,...,group_315,group_381,group_426,group_4770,group_8913,group_9224,group_9493,group_9494,group_9905,pheno
0,107,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.871375
1,109,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.364375
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0.803000
3,120335,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.952625
4,120337,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1.012875


In [165]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [166]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [167]:
df['pheno']
df['pheno'].value_counts()

1    135
0    120
Name: pheno, dtype: int64

In [168]:
df.shape

(255, 7004)

In [169]:
df_clean = df.drop(columns=['id'])

In [170]:
df_clean.shape

(255, 7003)

In [171]:
df_clean.head()

,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTGTAAATCTTTTCCATTTATTTCTTCGCTTTTCTCGCTACAATTTACAAATGGACTTGTTATCAACGTTAGGGCAATAAATGAAAGTAGTATAA,TTTTTTTGATAATTT,TTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAAAAAACGTATTTAATGTTGACAATACGAAAATTTCGTATTATATTAGGT,TTTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCA,TTTTTTTATTTTGCAATTTTTTATTTTCATTATAAACTTCCTTTCAAACACTGCTGAAATAGACGTCTTTTTCAAATAAGCATGATTAATACTTCAATTC,TTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAGT,TTTTTTTATGTTATTATAATTAAAGTTTTCCATTGTTTTCCTCCTATAATAGCTTATCTGCAATCATCACAGCTAATAAATCGTTTTGTCTTATTGCTTC,TTTTTTTATGTTATAATCTTTCTAGACGTATTCAAAGGACGTCTTTTTAGATTGTATGTTATAGCTAGCCTTCCGGTTAATTTTTTGTTATGATGTGTTA,TTTTTTTATAACATTCAAAGTCTCACCATTGTCATTTGAATGATCATCAATAATAATTAATTCGTAATCAGTACTCTTCATTGTTTGATTTAATACAGAA,...,group_315,group_381,group_426,group_4770,group_8913,group_9224,group_9493,group_9494,group_9905,pheno
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [172]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 7002) (255,)


In [173]:
# split into train, validation, and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y)

In [174]:
############# Fully-Connected Neural Network ################

In [220]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [225]:
#### neural network
model1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [226]:
model1.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [227]:
model1.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 178 samples, validate on 77 samples
Epoch 1/100
178/178 [==============================] - 0s 732us/step - loss: 0.6496 - accuracy: 0.6292 - val_loss: 0.6368 - val_accuracy: 0.5844
Epoch 2/100
178/178 [==============================] - 0s 268us/step - loss: 0.5966 - accuracy: 0.6461 - val_loss: 0.6314 - val_accuracy: 0.5844
Epoch 3/100
178/178 [==============================] - 0s 302us/step - loss: 0.5689 - accuracy: 0.6798 - val_loss: 0.6370 - val_accuracy: 0.6234
Epoch 4/100
178/178 [==============================] - 0s 293us/step - loss: 0.5451 - accuracy: 0.7135 - val_loss: 0.6336 - val_accuracy: 0.6234
Epoch 5/100
178/178 [==============================] - 0s 294us/step - loss: 0.5315 - accuracy: 0.7472 - val_loss: 0.6315 - val_accuracy: 0.6753
Epoch 6/100
178/178 [==============================] - 0s 235us/step - loss: 0.5155 - accuracy: 0.7472 - val_loss: 0.6325 - val_accuracy: 0.6234
Epoch 7/100
178/178 [==============================] - 0s 228us/step - loss: 0.5050 -

Epoch 57/100
178/178 [==============================] - 0s 246us/step - loss: 0.3272 - accuracy: 0.8820 - val_loss: 0.7292 - val_accuracy: 0.6234
Epoch 58/100
178/178 [==============================] - 0s 206us/step - loss: 0.3279 - accuracy: 0.8820 - val_loss: 0.7450 - val_accuracy: 0.6364
Epoch 59/100
178/178 [==============================] - 0s 195us/step - loss: 0.3239 - accuracy: 0.8876 - val_loss: 0.7353 - val_accuracy: 0.6364
Epoch 60/100
178/178 [==============================] - 0s 202us/step - loss: 0.3214 - accuracy: 0.8876 - val_loss: 0.7392 - val_accuracy: 0.6364
Epoch 61/100
178/178 [==============================] - 0s 198us/step - loss: 0.3203 - accuracy: 0.8876 - val_loss: 0.7493 - val_accuracy: 0.6234
Epoch 62/100
178/178 [==============================] - 0s 237us/step - loss: 0.3185 - accuracy: 0.8820 - val_loss: 0.7382 - val_accuracy: 0.6494
Epoch 63/100
178/178 [==============================] - 0s 233us/step - loss: 0.3181 - accuracy: 0.8820 - val_loss: 0.7490 -

In [228]:
acc_test = model1.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test*100))

77/77 [==============================] - 0s 197us/step
test accuracy: 64.94%


In [233]:
#### improve neural network
model2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [234]:
model2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [235]:
model2.fit(X_train, y_train,
          batch_size=64, epochs=1000,
          validation_data=(X_test, y_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 829us/step - loss: 0.6769 - accuracy: 0.5618 - val_loss: 0.6516 - val_accuracy: 0.5844
Epoch 2/1000
178/178 [==============================] - 0s 204us/step - loss: 0.6415 - accuracy: 0.6629 - val_loss: 0.6378 - val_accuracy: 0.5844
Epoch 3/1000
178/178 [==============================] - 0s 234us/step - loss: 0.6176 - accuracy: 0.6404 - val_loss: 0.6359 - val_accuracy: 0.5714
Epoch 4/1000
178/178 [==============================] - 0s 216us/step - loss: 0.6041 - accuracy: 0.6348 - val_loss: 0.6341 - val_accuracy: 0.5455
Epoch 5/1000
178/178 [==============================] - 0s 211us/step - loss: 0.5927 - accuracy: 0.6742 - val_loss: 0.6359 - val_accuracy: 0.5584
Epoch 6/1000
178/178 [==============================] - 0s 205us/step - loss: 0.5834 - accuracy: 0.6742 - val_loss: 0.6359 - val_accuracy: 0.5714
Epoch 7/1000
178/178 [==============================] - 0s 245us/step - loss: 0

178/178 [==============================] - 0s 220us/step - loss: 0.3980 - accuracy: 0.8371 - val_loss: 0.6891 - val_accuracy: 0.6234
Epoch 57/1000
178/178 [==============================] - 0s 198us/step - loss: 0.3944 - accuracy: 0.8315 - val_loss: 0.6887 - val_accuracy: 0.6234
Epoch 58/1000
178/178 [==============================] - 0s 222us/step - loss: 0.3931 - accuracy: 0.8315 - val_loss: 0.6922 - val_accuracy: 0.6234
Epoch 59/1000
178/178 [==============================] - 0s 179us/step - loss: 0.3915 - accuracy: 0.8427 - val_loss: 0.6910 - val_accuracy: 0.6364
Epoch 60/1000
178/178 [==============================] - 0s 194us/step - loss: 0.3918 - accuracy: 0.8258 - val_loss: 0.6931 - val_accuracy: 0.6104
Epoch 61/1000
178/178 [==============================] - 0s 232us/step - loss: 0.3871 - accuracy: 0.8315 - val_loss: 0.6919 - val_accuracy: 0.6104
Epoch 62/1000
178/178 [==============================] - 0s 210us/step - loss: 0.3870 - accuracy: 0.8371 - val_loss: 0.6909 - val_ac

Epoch 112/1000
178/178 [==============================] - 0s 177us/step - loss: 0.3112 - accuracy: 0.9101 - val_loss: 0.7652 - val_accuracy: 0.6494
Epoch 113/1000
178/178 [==============================] - 0s 158us/step - loss: 0.3075 - accuracy: 0.9101 - val_loss: 0.7649 - val_accuracy: 0.6494
Epoch 114/1000
178/178 [==============================] - 0s 185us/step - loss: 0.3049 - accuracy: 0.9157 - val_loss: 0.7653 - val_accuracy: 0.6364
Epoch 115/1000
178/178 [==============================] - 0s 165us/step - loss: 0.3045 - accuracy: 0.8989 - val_loss: 0.7696 - val_accuracy: 0.6494
Epoch 116/1000
178/178 [==============================] - 0s 176us/step - loss: 0.3043 - accuracy: 0.9157 - val_loss: 0.7677 - val_accuracy: 0.6494
Epoch 117/1000
178/178 [==============================] - 0s 209us/step - loss: 0.3050 - accuracy: 0.8989 - val_loss: 0.7754 - val_accuracy: 0.6494
Epoch 118/1000
178/178 [==============================] - 0s 194us/step - loss: 0.3036 - accuracy: 0.8989 - val_

178/178 [==============================] - 0s 166us/step - loss: 0.2478 - accuracy: 0.9213 - val_loss: 0.8589 - val_accuracy: 0.6364
Epoch 168/1000
178/178 [==============================] - 0s 196us/step - loss: 0.2445 - accuracy: 0.9157 - val_loss: 0.8628 - val_accuracy: 0.6364
Epoch 169/1000
178/178 [==============================] - 0s 182us/step - loss: 0.2439 - accuracy: 0.9157 - val_loss: 0.8671 - val_accuracy: 0.6234
Epoch 170/1000
178/178 [==============================] - 0s 163us/step - loss: 0.2425 - accuracy: 0.9213 - val_loss: 0.8668 - val_accuracy: 0.6364
Epoch 171/1000
178/178 [==============================] - 0s 161us/step - loss: 0.2437 - accuracy: 0.9213 - val_loss: 0.8742 - val_accuracy: 0.6494
Epoch 172/1000
178/178 [==============================] - 0s 213us/step - loss: 0.2430 - accuracy: 0.9213 - val_loss: 0.8728 - val_accuracy: 0.6364
Epoch 173/1000
178/178 [==============================] - 0s 161us/step - loss: 0.2454 - accuracy: 0.9213 - val_loss: 0.8703 - 

178/178 [==============================] - 0s 195us/step - loss: 0.2022 - accuracy: 0.9270 - val_loss: 0.9731 - val_accuracy: 0.6494
Epoch 223/1000
178/178 [==============================] - 0s 187us/step - loss: 0.1991 - accuracy: 0.9270 - val_loss: 0.9740 - val_accuracy: 0.6234
Epoch 224/1000
178/178 [==============================] - 0s 178us/step - loss: 0.1963 - accuracy: 0.9326 - val_loss: 0.9794 - val_accuracy: 0.6494
Epoch 225/1000
178/178 [==============================] - 0s 169us/step - loss: 0.1983 - accuracy: 0.9326 - val_loss: 0.9814 - val_accuracy: 0.6364
Epoch 226/1000
178/178 [==============================] - ETA: 0s - loss: 0.1389 - accuracy: 0.98 - 0s 169us/step - loss: 0.1973 - accuracy: 0.9326 - val_loss: 0.9812 - val_accuracy: 0.6494
Epoch 227/1000
178/178 [==============================] - 0s 168us/step - loss: 0.1958 - accuracy: 0.9382 - val_loss: 0.9798 - val_accuracy: 0.6364
Epoch 228/1000
178/178 [==============================] - 0s 183us/step - loss: 0.194

...

178/178 [==============================] - 0s 175us/step - loss: 0.0509 - accuracy: 0.9888 - val_loss: 1.8742 - val_accuracy: 0.6494
Epoch 773/1000
178/178 [==============================] - 0s 186us/step - loss: 0.0529 - accuracy: 0.9831 - val_loss: 1.8727 - val_accuracy: 0.6494
Epoch 774/1000
178/178 [==============================] - 0s 191us/step - loss: 0.0438 - accuracy: 0.9944 - val_loss: 1.8991 - val_accuracy: 0.6623
Epoch 775/1000
178/178 [==============================] - 0s 160us/step - loss: 0.0433 - accuracy: 0.9888 - val_loss: 1.8854 - val_accuracy: 0.6494
Epoch 776/1000
178/178 [==============================] - 0s 171us/step - loss: 0.0432 - accuracy: 0.9944 - val_loss: 1.8881 - val_accuracy: 0.6234
Epoch 777/1000
178/178 [==============================] - 0s 222us/step - loss: 0.0500 - accuracy: 0.9888 - val_loss: 1.8969 - val_accuracy: 0.6753
Epoch 778/1000
178/178 [==============================] - 0s 175us/step - loss: 0.0487 - accuracy: 0.9888 - val_loss: 1.8972 - 

178/178 [==============================] - 0s 188us/step - loss: 0.0385 - accuracy: 0.9831 - val_loss: 1.9356 - val_accuracy: 0.6364
Epoch 828/1000
178/178 [==============================] - 0s 195us/step - loss: 0.0374 - accuracy: 1.0000 - val_loss: 1.9593 - val_accuracy: 0.6623
Epoch 829/1000
178/178 [==============================] - 0s 177us/step - loss: 0.0403 - accuracy: 0.9944 - val_loss: 1.9406 - val_accuracy: 0.6234
Epoch 830/1000
178/178 [==============================] - 0s 160us/step - loss: 0.0386 - accuracy: 0.9944 - val_loss: 1.9447 - val_accuracy: 0.6364
Epoch 831/1000
178/178 [==============================] - 0s 230us/step - loss: 0.0406 - accuracy: 0.9831 - val_loss: 1.9482 - val_accuracy: 0.6753
Epoch 832/1000
178/178 [==============================] - 0s 219us/step - loss: 0.0376 - accuracy: 0.9944 - val_loss: 1.9488 - val_accuracy: 0.6364
Epoch 833/1000
178/178 [==============================] - 0s 249us/step - loss: 0.0411 - accuracy: 0.9944 - val_loss: 1.9674 - 

178/178 [==============================] - 0s 160us/step - loss: 0.0384 - accuracy: 0.9888 - val_loss: 2.0018 - val_accuracy: 0.6494
Epoch 883/1000
178/178 [==============================] - 0s 171us/step - loss: 0.0357 - accuracy: 0.9944 - val_loss: 1.9961 - val_accuracy: 0.6364
Epoch 884/1000
178/178 [==============================] - 0s 207us/step - loss: 0.0336 - accuracy: 1.0000 - val_loss: 2.0128 - val_accuracy: 0.6623
Epoch 885/1000
178/178 [==============================] - 0s 157us/step - loss: 0.0401 - accuracy: 0.9944 - val_loss: 1.9987 - val_accuracy: 0.6234
Epoch 886/1000
178/178 [==============================] - 0s 197us/step - loss: 0.0381 - accuracy: 0.9944 - val_loss: 2.0189 - val_accuracy: 0.6494
Epoch 887/1000
178/178 [==============================] - 0s 164us/step - loss: 0.0390 - accuracy: 0.9775 - val_loss: 2.0140 - val_accuracy: 0.6494
Epoch 888/1000
178/178 [==============================] - 0s 185us/step - loss: 0.0385 - accuracy: 0.9888 - val_loss: 2.0100 - 

178/178 [==============================] - 0s 187us/step - loss: 0.0350 - accuracy: 0.9831 - val_loss: 2.0457 - val_accuracy: 0.6494
Epoch 938/1000
178/178 [==============================] - 0s 192us/step - loss: 0.0345 - accuracy: 0.9944 - val_loss: 2.0605 - val_accuracy: 0.6753
Epoch 939/1000
178/178 [==============================] - 0s 179us/step - loss: 0.0362 - accuracy: 0.9944 - val_loss: 2.0606 - val_accuracy: 0.6623
Epoch 940/1000
178/178 [==============================] - 0s 162us/step - loss: 0.0335 - accuracy: 0.9944 - val_loss: 2.0700 - val_accuracy: 0.6494
Epoch 941/1000
178/178 [==============================] - 0s 186us/step - loss: 0.0305 - accuracy: 0.9944 - val_loss: 2.0568 - val_accuracy: 0.6494
Epoch 942/1000
178/178 [==============================] - 0s 167us/step - loss: 0.0332 - accuracy: 0.9944 - val_loss: 2.0626 - val_accuracy: 0.6364
Epoch 943/1000
178/178 [==============================] - 0s 165us/step - loss: 0.0342 - accuracy: 0.9944 - val_loss: 2.0725 - 

178/178 [==============================] - 0s 175us/step - loss: 0.0278 - accuracy: 0.9888 - val_loss: 2.1026 - val_accuracy: 0.6494
Epoch 993/1000
178/178 [==============================] - 0s 171us/step - loss: 0.0288 - accuracy: 1.0000 - val_loss: 2.1329 - val_accuracy: 0.6494
Epoch 994/1000
178/178 [==============================] - 0s 190us/step - loss: 0.0322 - accuracy: 0.9944 - val_loss: 2.1225 - val_accuracy: 0.6494
Epoch 995/1000
178/178 [==============================] - 0s 175us/step - loss: 0.0293 - accuracy: 0.9944 - val_loss: 2.1081 - val_accuracy: 0.6494
Epoch 996/1000
178/178 [==============================] - 0s 156us/step - loss: 0.0317 - accuracy: 0.9944 - val_loss: 2.1382 - val_accuracy: 0.6494
Epoch 997/1000
178/178 [==============================] - 0s 206us/step - loss: 0.0325 - accuracy: 0.9831 - val_loss: 2.1395 - val_accuracy: 0.6623
Epoch 998/1000
178/178 [==============================] - 0s 177us/step - loss: 0.0292 - accuracy: 0.9944 - val_loss: 2.1219 - 

In [236]:
acc_test2 = model2.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test2*100))

77/77 [==============================] - 0s 200us/step
test accuracy: 66.23%


In [237]:
#### add regularizer and dropout
model3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [238]:
model3.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [239]:
model3.fit(X_train, y_train,
          batch_size=64, epochs=1000,
          validation_data=(X_test, y_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 1ms/step - loss: 2.6316 - accuracy: 0.4944 - val_loss: 0.8442 - val_accuracy: 0.5065
Epoch 2/1000
178/178 [==============================] - 0s 215us/step - loss: 2.4840 - accuracy: 0.4944 - val_loss: 0.7591 - val_accuracy: 0.5195
Epoch 3/1000
178/178 [==============================] - 0s 237us/step - loss: 2.4473 - accuracy: 0.4775 - val_loss: 0.7221 - val_accuracy: 0.5455
Epoch 4/1000
178/178 [==============================] - 0s 222us/step - loss: 2.0508 - accuracy: 0.5393 - val_loss: 0.7071 - val_accuracy: 0.5714
Epoch 5/1000
178/178 [==============================] - 0s 258us/step - loss: 1.7281 - accuracy: 0.5337 - val_loss: 0.7038 - val_accuracy: 0.5844
Epoch 6/1000
178/178 [==============================] - 0s 251us/step - loss: 2.3961 - accuracy: 0.4888 - val_loss: 0.6987 - val_accuracy: 0.5974
Epoch 7/1000
178/178 [==============================] - 0s 197us/step - loss: 2.3

178/178 [==============================] - 0s 240us/step - loss: 2.7247 - accuracy: 0.6292 - val_loss: 0.6976 - val_accuracy: 0.6623
Epoch 57/1000
178/178 [==============================] - 0s 262us/step - loss: 2.3820 - accuracy: 0.6404 - val_loss: 0.6970 - val_accuracy: 0.6623
Epoch 58/1000
178/178 [==============================] - 0s 228us/step - loss: 2.1211 - accuracy: 0.6629 - val_loss: 0.6987 - val_accuracy: 0.6623
Epoch 59/1000
178/178 [==============================] - 0s 230us/step - loss: 2.4930 - accuracy: 0.6180 - val_loss: 0.6989 - val_accuracy: 0.6494
Epoch 60/1000
178/178 [==============================] - 0s 206us/step - loss: 1.9922 - accuracy: 0.6461 - val_loss: 0.6986 - val_accuracy: 0.6623
Epoch 61/1000
178/178 [==============================] - 0s 208us/step - loss: 2.0606 - accuracy: 0.6517 - val_loss: 0.6993 - val_accuracy: 0.6623
Epoch 62/1000
178/178 [==============================] - 0s 219us/step - loss: 1.9661 - accuracy: 0.6685 - val_loss: 0.6985 - val_ac

Epoch 112/1000
178/178 [==============================] - 0s 283us/step - loss: 2.0284 - accuracy: 0.7079 - val_loss: 0.6977 - val_accuracy: 0.6623
Epoch 113/1000
178/178 [==============================] - 0s 343us/step - loss: 1.7691 - accuracy: 0.7416 - val_loss: 0.6997 - val_accuracy: 0.6623
Epoch 114/1000
178/178 [==============================] - 0s 417us/step - loss: 1.6583 - accuracy: 0.7528 - val_loss: 0.6951 - val_accuracy: 0.6753
Epoch 115/1000
178/178 [==============================] - 0s 310us/step - loss: 2.1154 - accuracy: 0.7191 - val_loss: 0.6968 - val_accuracy: 0.6623
Epoch 116/1000
178/178 [==============================] - 0s 241us/step - loss: 1.8407 - accuracy: 0.7416 - val_loss: 0.6984 - val_accuracy: 0.6623
Epoch 117/1000
178/178 [==============================] - 0s 251us/step - loss: 2.0542 - accuracy: 0.7753 - val_loss: 0.6951 - val_accuracy: 0.6753
Epoch 118/1000
178/178 [==============================] - 0s 185us/step - loss: 2.5237 - accuracy: 0.7191 - val_

178/178 [==============================] - 0s 409us/step - loss: 2.3872 - accuracy: 0.7528 - val_loss: 0.7096 - val_accuracy: 0.6494
Epoch 168/1000
178/178 [==============================] - 0s 601us/step - loss: 1.8006 - accuracy: 0.7978 - val_loss: 0.7104 - val_accuracy: 0.6623
Epoch 169/1000
178/178 [==============================] - 0s 368us/step - loss: 2.5331 - accuracy: 0.7809 - val_loss: 0.7098 - val_accuracy: 0.6623
Epoch 170/1000
178/178 [==============================] - 0s 187us/step - loss: 2.5605 - accuracy: 0.7528 - val_loss: 0.7115 - val_accuracy: 0.6623
Epoch 171/1000
178/178 [==============================] - 0s 231us/step - loss: 2.1172 - accuracy: 0.7865 - val_loss: 0.7102 - val_accuracy: 0.6623
Epoch 172/1000
178/178 [==============================] - 0s 204us/step - loss: 1.8579 - accuracy: 0.7978 - val_loss: 0.7114 - val_accuracy: 0.6364
Epoch 173/1000
178/178 [==============================] - 0s 179us/step - loss: 1.9551 - accuracy: 0.7978 - val_loss: 0.7164 - 

178/178 [==============================] - 0s 203us/step - loss: 1.4270 - accuracy: 0.8371 - val_loss: 0.7261 - val_accuracy: 0.6364
Epoch 223/1000
178/178 [==============================] - 0s 209us/step - loss: 2.0181 - accuracy: 0.7921 - val_loss: 0.7287 - val_accuracy: 0.6364
Epoch 224/1000
178/178 [==============================] - 0s 181us/step - loss: 1.9399 - accuracy: 0.7978 - val_loss: 0.7273 - val_accuracy: 0.6364
Epoch 225/1000
178/178 [==============================] - 0s 253us/step - loss: 2.0923 - accuracy: 0.7921 - val_loss: 0.7256 - val_accuracy: 0.6364
Epoch 226/1000
178/178 [==============================] - 0s 259us/step - loss: 2.2519 - accuracy: 0.7921 - val_loss: 0.7262 - val_accuracy: 0.6364
Epoch 227/1000
178/178 [==============================] - 0s 258us/step - loss: 1.4857 - accuracy: 0.8427 - val_loss: 0.7316 - val_accuracy: 0.6364
Epoch 228/1000
178/178 [==============================] - 0s 225us/step - loss: 1.9213 - accuracy: 0.8090 - val_loss: 0.7280 - 

...

178/178 [==============================] - 0s 219us/step - loss: 1.7845 - accuracy: 0.8596 - val_loss: 0.8181 - val_accuracy: 0.6234
Epoch 772/1000
178/178 [==============================] - 0s 202us/step - loss: 2.2267 - accuracy: 0.8202 - val_loss: 0.8195 - val_accuracy: 0.6364
Epoch 773/1000
178/178 [==============================] - 0s 183us/step - loss: 1.9794 - accuracy: 0.8258 - val_loss: 0.8191 - val_accuracy: 0.6494
Epoch 774/1000
178/178 [==============================] - 0s 218us/step - loss: 2.1152 - accuracy: 0.8315 - val_loss: 0.8147 - val_accuracy: 0.6494
Epoch 775/1000
178/178 [==============================] - 0s 190us/step - loss: 1.8796 - accuracy: 0.8315 - val_loss: 0.8059 - val_accuracy: 0.6494
Epoch 776/1000
178/178 [==============================] - 0s 193us/step - loss: 1.6000 - accuracy: 0.8708 - val_loss: 0.8225 - val_accuracy: 0.6494
Epoch 777/1000
178/178 [==============================] - 0s 184us/step - loss: 1.5407 - accuracy: 0.8539 - val_loss: 0.8175 - 

178/178 [==============================] - 0s 222us/step - loss: 2.1285 - accuracy: 0.8258 - val_loss: 0.8234 - val_accuracy: 0.6494
Epoch 827/1000
178/178 [==============================] - 0s 331us/step - loss: 1.7706 - accuracy: 0.8483 - val_loss: 0.8161 - val_accuracy: 0.6494
Epoch 828/1000
178/178 [==============================] - 0s 205us/step - loss: 2.1123 - accuracy: 0.8258 - val_loss: 0.8118 - val_accuracy: 0.6364
Epoch 829/1000
178/178 [==============================] - 0s 162us/step - loss: 2.1160 - accuracy: 0.8315 - val_loss: 0.8240 - val_accuracy: 0.6234
Epoch 830/1000
178/178 [==============================] - 0s 182us/step - loss: 2.1124 - accuracy: 0.8315 - val_loss: 0.8333 - val_accuracy: 0.6364
Epoch 831/1000
178/178 [==============================] - 0s 397us/step - loss: 1.3562 - accuracy: 0.8764 - val_loss: 0.8189 - val_accuracy: 0.6494
Epoch 832/1000
178/178 [==============================] - 0s 169us/step - loss: 2.2832 - accuracy: 0.8258 - val_loss: 0.8283 - 

178/178 [==============================] - 0s 205us/step - loss: 1.8420 - accuracy: 0.8539 - val_loss: 0.8206 - val_accuracy: 0.6234
Epoch 882/1000
178/178 [==============================] - 0s 197us/step - loss: 1.6032 - accuracy: 0.8708 - val_loss: 0.8195 - val_accuracy: 0.6234
Epoch 883/1000
178/178 [==============================] - 0s 183us/step - loss: 1.8571 - accuracy: 0.8483 - val_loss: 0.8416 - val_accuracy: 0.6364
Epoch 884/1000
178/178 [==============================] - 0s 206us/step - loss: 1.7780 - accuracy: 0.8539 - val_loss: 0.8261 - val_accuracy: 0.6234
Epoch 885/1000
178/178 [==============================] - 0s 224us/step - loss: 1.3400 - accuracy: 0.8820 - val_loss: 0.8378 - val_accuracy: 0.6234
Epoch 886/1000
178/178 [==============================] - 0s 182us/step - loss: 1.7856 - accuracy: 0.8539 - val_loss: 0.8301 - val_accuracy: 0.6494
Epoch 887/1000
178/178 [==============================] - 0s 197us/step - loss: 1.8589 - accuracy: 0.8483 - val_loss: 0.8305 - 

178/178 [==============================] - 0s 165us/step - loss: 1.5046 - accuracy: 0.8764 - val_loss: 0.8291 - val_accuracy: 0.6234
Epoch 937/1000
178/178 [==============================] - 0s 208us/step - loss: 1.9545 - accuracy: 0.8315 - val_loss: 0.8479 - val_accuracy: 0.6364
Epoch 938/1000
178/178 [==============================] - 0s 186us/step - loss: 2.0357 - accuracy: 0.8315 - val_loss: 0.8339 - val_accuracy: 0.6364
Epoch 939/1000
178/178 [==============================] - 0s 179us/step - loss: 1.1585 - accuracy: 0.8989 - val_loss: 0.8460 - val_accuracy: 0.6234
Epoch 940/1000
178/178 [==============================] - 0s 174us/step - loss: 2.3588 - accuracy: 0.8202 - val_loss: 0.8461 - val_accuracy: 0.6364
Epoch 941/1000
178/178 [==============================] - 0s 187us/step - loss: 1.7584 - accuracy: 0.8539 - val_loss: 0.8618 - val_accuracy: 0.6364
Epoch 942/1000
178/178 [==============================] - 0s 188us/step - loss: 1.5259 - accuracy: 0.8596 - val_loss: 0.8386 - 

178/178 [==============================] - 0s 172us/step - loss: 1.6634 - accuracy: 0.8652 - val_loss: 0.8630 - val_accuracy: 0.6494
Epoch 992/1000
178/178 [==============================] - 0s 178us/step - loss: 1.7535 - accuracy: 0.8652 - val_loss: 0.8488 - val_accuracy: 0.6364
Epoch 993/1000
178/178 [==============================] - 0s 265us/step - loss: 2.8006 - accuracy: 0.7921 - val_loss: 0.8480 - val_accuracy: 0.6234
Epoch 994/1000
178/178 [==============================] - 0s 274us/step - loss: 1.5773 - accuracy: 0.8652 - val_loss: 0.8524 - val_accuracy: 0.6234
Epoch 995/1000
178/178 [==============================] - 0s 208us/step - loss: 1.7604 - accuracy: 0.8596 - val_loss: 0.8729 - val_accuracy: 0.6364
Epoch 996/1000
178/178 [==============================] - 0s 209us/step - loss: 1.7640 - accuracy: 0.8539 - val_loss: 0.8480 - val_accuracy: 0.6364
Epoch 997/1000
178/178 [==============================] - 0s 225us/step - loss: 2.4599 - accuracy: 0.8090 - val_loss: 0.8476 - 

In [240]:
acc_test3 = model3.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test3*100))

77/77 [==============================] - 0s 170us/step
test accuracy: 62.34%


In [241]:
############ Feature selection using lasso ##########

In [242]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [243]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
selection.fit(X_train, y_train)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [244]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [245]:
X_train_features = np.vstack((names, X_train))
X_train_features = pd.DataFrame(X_train_features)

In [246]:
sel_features = X_train_features.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 7002
selected features: 119


In [247]:
cols = sel_features.values
cols.reshape((1, -1))

array([[ 354,  403,  527,  633,  634,  647,  672,  861,  874,  883,  994,
         998, 1290, 1372, 1402, 1449, 1521, 1537, 1559, 1561, 1576, 1606,
        1663, 1817, 1899, 1947, 2089, 2106, 2117, 2235, 2283, 2292, 2370,
        2511, 2520, 2546, 2587, 2630, 2666, 2830, 2997, 3013, 3046, 3109,
        3173, 3391, 3532, 3584, 3586, 3703, 3752, 3817, 3874, 3882, 4031,
        4047, 4096, 4227, 4278, 4331, 4344, 4345, 4350, 4449, 4463, 4544,
        4545, 4605, 4665, 4736, 4738, 4762, 4842, 5047, 5068, 5100, 5128,
        5189, 5382, 5487, 5658, 5729, 5745, 5758, 5799, 5824, 5825, 5890,
        5955, 6005, 6067, 6109, 6187, 6192, 6215, 6285, 6393, 6421, 6424,
        6533, 6547, 6581, 6596, 6616, 6745, 6914, 6926, 6933, 6935, 6941,
        6960, 6965, 6972, 6976, 6977, 6980, 6983, 6984, 6986]])

In [248]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTGGAACAAAAC',
       'TTTGAATGCTAACGTAACTATTTCCATCTCTTCGTTTAACTAAGTTGAATTTGCCCATACCTGATTCAATTGTAGTGTCTCCGCCTGTAGCATAGTCACC',
       'TTTCAAGCGTCATTTCAATAAAACGCTGGTCTATCATTAAGTCGTAGCCATCGTTGTATTGAATATTAACGGGTCGTCTATTACATTCTTCGTCATAGTA',
       'TTTAGACTTAAT', 'TTTAGACTTAATCG',
       'TTTACTGCCGTCATCTTGCCAAAACGTTTCGTAACATGTTCAATTACAAGTCCCATACTTTGCCTCCTAAAAAAATATGTATTTATCTTAATATAACATT',
       'TTTACAATTGCTTATAAGAAATCTGAAACATTTAGAAATTTTGTTAATGGTGCAATTGAAAGTGTTAAACAAACATTTAGTAATTTTATTCAATTTATTC',
       'TTGGAGTAATT',
       'TTGCTTATAAGAAATCTGAAACATTTAGAAATTTTGTTAATGGTGCAATTGAAAGTGTTAAACAAACATTTAGTAATTTTATTCAATTTATTCAACCTTT',
       'TTGCTACTATAA',
       'TTGAATCAGGTATGGGCAAATTCAACTTAGTTAAACGAAGAGATGGAAATAGTTACGTTAGCATTCAAAGTTATGATTTATTGGCGGTAGGTTCTGATAA',
       'TTGAATAAATTGAATAAAATTACTAAATGTTTGTTTAACACTTTCAATTGCACCATTAACAAAATTTCTAAATGTTTCAGATTTCTTATAAGCAATTGTA',
       'TTATTTTATTACTA', 'TTATCCGAAATTT', 'TTATAGTAGCA',
       'TTAGTTCCTACGAATAAGACTTGTCCACCATCTTCTGAAACTTGTTTC

In [249]:
removed_features = X_train_features.columns[(selection.estimator_.coef_ == 0).ravel().tolist()]
removed_features

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            6992, 6993, 6994, 6995, 6996, 6997, 6998, 6999, 7000, 7001],
           dtype='int64', length=6883)

In [250]:
len(removed_features)

6883

In [251]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']
X_sel = df_sel.loc[:, df_sel.columns != 'pheno'].values
y_sel = df_sel['pheno'].values
print(X_sel.shape, y_sel.shape, df_sel.shape)

(255, 119) (255,) (255, 120)


In [252]:
# split into train, validation, and test data
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel)

In [253]:
#### neural network
model_sel1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [254]:
model_sel1.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [255]:
model_sel1.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 625us/step - loss: 0.6722 - accuracy: 0.5281 - val_loss: 0.7150 - val_accuracy: 0.5325
Epoch 2/1000
178/178 [==============================] - 0s 104us/step - loss: 0.6618 - accuracy: 0.5281 - val_loss: 0.7041 - val_accuracy: 0.5325
Epoch 3/1000
178/178 [==============================] - 0s 151us/step - loss: 0.6522 - accuracy: 0.5281 - val_loss: 0.6969 - val_accuracy: 0.5325
Epoch 4/1000
178/178 [==============================] - 0s 137us/step - loss: 0.6448 - accuracy: 0.5281 - val_loss: 0.6904 - val_accuracy: 0.5325
Epoch 5/1000
178/178 [==============================] - 0s 126us/step - loss: 0.6385 - accuracy: 0.5337 - val_loss: 0.6852 - val_accuracy: 0.5455
Epoch 6/1000
178/178 [==============================] - 0s 172us/step - loss: 0.6322 - accuracy: 0.5506 - val_loss: 0.6817 - val_accuracy: 0.5455
Epoch 7/1000
178/178 [==============================] - 0s 175us/step - loss: 0

178/178 [==============================] - 0s 109us/step - loss: 0.4740 - accuracy: 0.7978 - val_loss: 0.6315 - val_accuracy: 0.6104
Epoch 57/1000
178/178 [==============================] - 0s 168us/step - loss: 0.4719 - accuracy: 0.7978 - val_loss: 0.6313 - val_accuracy: 0.6104
Epoch 58/1000
178/178 [==============================] - 0s 123us/step - loss: 0.4701 - accuracy: 0.7978 - val_loss: 0.6304 - val_accuracy: 0.6104
Epoch 59/1000
178/178 [==============================] - 0s 117us/step - loss: 0.4676 - accuracy: 0.7978 - val_loss: 0.6305 - val_accuracy: 0.6104
Epoch 60/1000
178/178 [==============================] - 0s 107us/step - loss: 0.4658 - accuracy: 0.8034 - val_loss: 0.6309 - val_accuracy: 0.6104
Epoch 61/1000
178/178 [==============================] - 0s 107us/step - loss: 0.4635 - accuracy: 0.7978 - val_loss: 0.6310 - val_accuracy: 0.6104
Epoch 62/1000
178/178 [==============================] - 0s 130us/step - loss: 0.4614 - accuracy: 0.8034 - val_loss: 0.6316 - val_ac

Epoch 112/1000
178/178 [==============================] - 0s 100us/step - loss: 0.3838 - accuracy: 0.8258 - val_loss: 0.6470 - val_accuracy: 0.6104
Epoch 113/1000
178/178 [==============================] - 0s 114us/step - loss: 0.3826 - accuracy: 0.8258 - val_loss: 0.6494 - val_accuracy: 0.6623
Epoch 114/1000
178/178 [==============================] - 0s 100us/step - loss: 0.3813 - accuracy: 0.8371 - val_loss: 0.6485 - val_accuracy: 0.6623
Epoch 115/1000
178/178 [==============================] - 0s 80us/step - loss: 0.3801 - accuracy: 0.8483 - val_loss: 0.6468 - val_accuracy: 0.6104
Epoch 116/1000
178/178 [==============================] - 0s 123us/step - loss: 0.3788 - accuracy: 0.8371 - val_loss: 0.6474 - val_accuracy: 0.6104
Epoch 117/1000
178/178 [==============================] - 0s 102us/step - loss: 0.3778 - accuracy: 0.8315 - val_loss: 0.6472 - val_accuracy: 0.6104
Epoch 118/1000
178/178 [==============================] - 0s 97us/step - loss: 0.3767 - accuracy: 0.8371 - val_lo

178/178 [==============================] - 0s 124us/step - loss: 0.3292 - accuracy: 0.8652 - val_loss: 0.6809 - val_accuracy: 0.6623
Epoch 168/1000
178/178 [==============================] - 0s 101us/step - loss: 0.3276 - accuracy: 0.8708 - val_loss: 0.6823 - val_accuracy: 0.6623
Epoch 169/1000
178/178 [==============================] - 0s 119us/step - loss: 0.3279 - accuracy: 0.8764 - val_loss: 0.6826 - val_accuracy: 0.6623
Epoch 170/1000
178/178 [==============================] - 0s 106us/step - loss: 0.3265 - accuracy: 0.8708 - val_loss: 0.6857 - val_accuracy: 0.6623
Epoch 171/1000
178/178 [==============================] - 0s 119us/step - loss: 0.3262 - accuracy: 0.8708 - val_loss: 0.6882 - val_accuracy: 0.6623
Epoch 172/1000
178/178 [==============================] - 0s 98us/step - loss: 0.3255 - accuracy: 0.8708 - val_loss: 0.6883 - val_accuracy: 0.6623
Epoch 173/1000
178/178 [==============================] - 0s 131us/step - loss: 0.3237 - accuracy: 0.8820 - val_loss: 0.6889 - v

178/178 [==============================] - 0s 105us/step - loss: 0.2903 - accuracy: 0.8876 - val_loss: 0.7270 - val_accuracy: 0.6623
Epoch 223/1000
178/178 [==============================] - 0s 112us/step - loss: 0.2891 - accuracy: 0.8876 - val_loss: 0.7286 - val_accuracy: 0.6623
Epoch 224/1000
178/178 [==============================] - 0s 113us/step - loss: 0.2883 - accuracy: 0.8876 - val_loss: 0.7307 - val_accuracy: 0.6364
Epoch 225/1000
178/178 [==============================] - 0s 99us/step - loss: 0.2883 - accuracy: 0.8820 - val_loss: 0.7310 - val_accuracy: 0.6623
Epoch 226/1000
178/178 [==============================] - 0s 107us/step - loss: 0.2875 - accuracy: 0.8876 - val_loss: 0.7324 - val_accuracy: 0.6623
Epoch 227/1000
178/178 [==============================] - 0s 103us/step - loss: 0.2872 - accuracy: 0.8876 - val_loss: 0.7364 - val_accuracy: 0.6364
Epoch 228/1000
178/178 [==============================] - 0s 102us/step - loss: 0.2861 - accuracy: 0.8820 - val_loss: 0.7352 - v

...

178/178 [==============================] - 0s 111us/step - loss: 0.1312 - accuracy: 0.9719 - val_loss: 1.2078 - val_accuracy: 0.6234
Epoch 773/1000
178/178 [==============================] - 0s 153us/step - loss: 0.1311 - accuracy: 0.9719 - val_loss: 1.2066 - val_accuracy: 0.6364
Epoch 774/1000
178/178 [==============================] - 0s 107us/step - loss: 0.1311 - accuracy: 0.9719 - val_loss: 1.2061 - val_accuracy: 0.6364
Epoch 775/1000
178/178 [==============================] - 0s 117us/step - loss: 0.1308 - accuracy: 0.9775 - val_loss: 1.2072 - val_accuracy: 0.6364
Epoch 776/1000
178/178 [==============================] - 0s 120us/step - loss: 0.1312 - accuracy: 0.9775 - val_loss: 1.2078 - val_accuracy: 0.6364
Epoch 777/1000
178/178 [==============================] - 0s 113us/step - loss: 0.1304 - accuracy: 0.9775 - val_loss: 1.2080 - val_accuracy: 0.6234
Epoch 778/1000
178/178 [==============================] - 0s 107us/step - loss: 0.1303 - accuracy: 0.9775 - val_loss: 1.2105 - 

178/178 [==============================] - 0s 98us/step - loss: 0.1251 - accuracy: 0.9775 - val_loss: 1.2421 - val_accuracy: 0.6234
Epoch 828/1000
178/178 [==============================] - 0s 135us/step - loss: 0.1250 - accuracy: 0.9775 - val_loss: 1.2439 - val_accuracy: 0.6234
Epoch 829/1000
178/178 [==============================] - 0s 108us/step - loss: 0.1247 - accuracy: 0.9775 - val_loss: 1.2463 - val_accuracy: 0.6234
Epoch 830/1000
178/178 [==============================] - 0s 109us/step - loss: 0.1252 - accuracy: 0.9775 - val_loss: 1.2460 - val_accuracy: 0.6234
Epoch 831/1000
178/178 [==============================] - 0s 110us/step - loss: 0.1246 - accuracy: 0.9775 - val_loss: 1.2451 - val_accuracy: 0.6234
Epoch 832/1000
178/178 [==============================] - 0s 106us/step - loss: 0.1248 - accuracy: 0.9775 - val_loss: 1.2474 - val_accuracy: 0.6234
Epoch 833/1000
178/178 [==============================] - 0s 101us/step - loss: 0.1246 - accuracy: 0.9775 - val_loss: 1.2473 - v

178/178 [==============================] - 0s 109us/step - loss: 0.1198 - accuracy: 0.9775 - val_loss: 1.2807 - val_accuracy: 0.6234
Epoch 883/1000
178/178 [==============================] - 0s 104us/step - loss: 0.1212 - accuracy: 0.9775 - val_loss: 1.2824 - val_accuracy: 0.6234
Epoch 884/1000
178/178 [==============================] - 0s 118us/step - loss: 0.1200 - accuracy: 0.9775 - val_loss: 1.2841 - val_accuracy: 0.6234
Epoch 885/1000
178/178 [==============================] - 0s 118us/step - loss: 0.1200 - accuracy: 0.9775 - val_loss: 1.2864 - val_accuracy: 0.6234
Epoch 886/1000
178/178 [==============================] - 0s 93us/step - loss: 0.1193 - accuracy: 0.9775 - val_loss: 1.2868 - val_accuracy: 0.6234
Epoch 887/1000
178/178 [==============================] - 0s 137us/step - loss: 0.1191 - accuracy: 0.9775 - val_loss: 1.2861 - val_accuracy: 0.6234
Epoch 888/1000
178/178 [==============================] - 0s 101us/step - loss: 0.1193 - accuracy: 0.9775 - val_loss: 1.2906 - v

178/178 [==============================] - 0s 124us/step - loss: 0.1154 - accuracy: 0.9775 - val_loss: 1.3206 - val_accuracy: 0.6234
Epoch 938/1000
178/178 [==============================] - 0s 145us/step - loss: 0.1148 - accuracy: 0.9775 - val_loss: 1.3225 - val_accuracy: 0.6234
Epoch 939/1000
178/178 [==============================] - 0s 132us/step - loss: 0.1150 - accuracy: 0.9775 - val_loss: 1.3224 - val_accuracy: 0.6234
Epoch 940/1000
178/178 [==============================] - 0s 125us/step - loss: 0.1152 - accuracy: 0.9775 - val_loss: 1.3211 - val_accuracy: 0.6494
Epoch 941/1000
178/178 [==============================] - 0s 107us/step - loss: 0.1151 - accuracy: 0.9775 - val_loss: 1.3186 - val_accuracy: 0.6494
Epoch 942/1000
178/178 [==============================] - 0s 133us/step - loss: 0.1145 - accuracy: 0.9775 - val_loss: 1.3238 - val_accuracy: 0.6234
Epoch 943/1000
178/178 [==============================] - 0s 223us/step - loss: 0.1146 - accuracy: 0.9775 - val_loss: 1.3263 - 

178/178 [==============================] - 0s 195us/step - loss: 0.1109 - accuracy: 0.9775 - val_loss: 1.3575 - val_accuracy: 0.6234
Epoch 993/1000
178/178 [==============================] - 0s 102us/step - loss: 0.1105 - accuracy: 0.9775 - val_loss: 1.3586 - val_accuracy: 0.6234
Epoch 994/1000
178/178 [==============================] - 0s 163us/step - loss: 0.1108 - accuracy: 0.9775 - val_loss: 1.3597 - val_accuracy: 0.6234
Epoch 995/1000
178/178 [==============================] - 0s 117us/step - loss: 0.1112 - accuracy: 0.9775 - val_loss: 1.3630 - val_accuracy: 0.6104
Epoch 996/1000
178/178 [==============================] - 0s 114us/step - loss: 0.1105 - accuracy: 0.9775 - val_loss: 1.3653 - val_accuracy: 0.6104
Epoch 997/1000
178/178 [==============================] - 0s 114us/step - loss: 0.1109 - accuracy: 0.9775 - val_loss: 1.3653 - val_accuracy: 0.6234
Epoch 998/1000
178/178 [==============================] - 0s 150us/step - loss: 0.1106 - accuracy: 0.9775 - val_loss: 1.3648 - 

In [256]:
acc_test_sel1 = model_sel1.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel1*100))

77/77 [==============================] - 0s 120us/step
test accuracy: 62.34%


In [257]:
#### improve neural network
model_sel2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [258]:
model_sel2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [259]:
model_sel2.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 720us/step - loss: 0.7146 - accuracy: 0.3315 - val_loss: 0.7138 - val_accuracy: 0.3247
Epoch 2/1000
178/178 [==============================] - 0s 98us/step - loss: 0.7047 - accuracy: 0.3989 - val_loss: 0.7057 - val_accuracy: 0.4675
Epoch 3/1000
178/178 [==============================] - 0s 151us/step - loss: 0.6957 - accuracy: 0.4326 - val_loss: 0.6984 - val_accuracy: 0.4545
Epoch 4/1000
178/178 [==============================] - 0s 158us/step - loss: 0.6876 - accuracy: 0.4944 - val_loss: 0.6924 - val_accuracy: 0.4675
Epoch 5/1000
178/178 [==============================] - 0s 145us/step - loss: 0.6802 - accuracy: 0.5225 - val_loss: 0.6869 - val_accuracy: 0.4935
Epoch 6/1000
178/178 [==============================] - 0s 132us/step - loss: 0.6731 - accuracy: 0.5618 - val_loss: 0.6826 - val_accuracy: 0.4805
Epoch 7/1000
178/178 [==============================] - 0s 150us/step - loss: 0.

Epoch 56/1000
178/178 [==============================] - 0s 236us/step - loss: 0.4987 - accuracy: 0.7697 - val_loss: 0.6460 - val_accuracy: 0.5844
Epoch 57/1000
178/178 [==============================] - 0s 201us/step - loss: 0.4960 - accuracy: 0.7753 - val_loss: 0.6459 - val_accuracy: 0.5844
Epoch 58/1000
178/178 [==============================] - 0s 195us/step - loss: 0.4933 - accuracy: 0.7753 - val_loss: 0.6451 - val_accuracy: 0.5844
Epoch 59/1000
178/178 [==============================] - 0s 118us/step - loss: 0.4909 - accuracy: 0.7753 - val_loss: 0.6457 - val_accuracy: 0.5844
Epoch 60/1000
178/178 [==============================] - 0s 136us/step - loss: 0.4878 - accuracy: 0.7753 - val_loss: 0.6459 - val_accuracy: 0.5844
Epoch 61/1000
178/178 [==============================] - 0s 104us/step - loss: 0.4851 - accuracy: 0.7809 - val_loss: 0.6450 - val_accuracy: 0.5844
Epoch 62/1000
178/178 [==============================] - 0s 111us/step - loss: 0.4825 - accuracy: 0.7753 - val_loss: 0

Epoch 112/1000
178/178 [==============================] - 0s 112us/step - loss: 0.3600 - accuracy: 0.8427 - val_loss: 0.6469 - val_accuracy: 0.6623
Epoch 113/1000
178/178 [==============================] - 0s 110us/step - loss: 0.3580 - accuracy: 0.8539 - val_loss: 0.6479 - val_accuracy: 0.6623
Epoch 114/1000
178/178 [==============================] - 0s 122us/step - loss: 0.3556 - accuracy: 0.8596 - val_loss: 0.6484 - val_accuracy: 0.6623
Epoch 115/1000
178/178 [==============================] - 0s 108us/step - loss: 0.3540 - accuracy: 0.8652 - val_loss: 0.6507 - val_accuracy: 0.6623
Epoch 116/1000
178/178 [==============================] - 0s 115us/step - loss: 0.3517 - accuracy: 0.8652 - val_loss: 0.6511 - val_accuracy: 0.6623
Epoch 117/1000
178/178 [==============================] - 0s 110us/step - loss: 0.3503 - accuracy: 0.8652 - val_loss: 0.6527 - val_accuracy: 0.6623
Epoch 118/1000
178/178 [==============================] - 0s 145us/step - loss: 0.3478 - accuracy: 0.8652 - val_

178/178 [==============================] - 0s 116us/step - loss: 0.2662 - accuracy: 0.9101 - val_loss: 0.7247 - val_accuracy: 0.6364
Epoch 168/1000
178/178 [==============================] - 0s 110us/step - loss: 0.2646 - accuracy: 0.9157 - val_loss: 0.7233 - val_accuracy: 0.6364
Epoch 169/1000
178/178 [==============================] - 0s 133us/step - loss: 0.2636 - accuracy: 0.9157 - val_loss: 0.7285 - val_accuracy: 0.6364
Epoch 170/1000
178/178 [==============================] - 0s 110us/step - loss: 0.2615 - accuracy: 0.9157 - val_loss: 0.7326 - val_accuracy: 0.6234
Epoch 171/1000
178/178 [==============================] - 0s 117us/step - loss: 0.2629 - accuracy: 0.9101 - val_loss: 0.7340 - val_accuracy: 0.6364
Epoch 172/1000
178/178 [==============================] - 0s 119us/step - loss: 0.2586 - accuracy: 0.9157 - val_loss: 0.7355 - val_accuracy: 0.6364
Epoch 173/1000
178/178 [==============================] - 0s 110us/step - loss: 0.2576 - accuracy: 0.9157 - val_loss: 0.7388 - 

178/178 [==============================] - 0s 150us/step - loss: 0.2067 - accuracy: 0.9494 - val_loss: 0.8457 - val_accuracy: 0.5974
Epoch 223/1000
178/178 [==============================] - 0s 108us/step - loss: 0.2064 - accuracy: 0.9494 - val_loss: 0.8474 - val_accuracy: 0.5974
Epoch 224/1000
178/178 [==============================] - 0s 98us/step - loss: 0.2049 - accuracy: 0.9494 - val_loss: 0.8527 - val_accuracy: 0.5974
Epoch 225/1000
178/178 [==============================] - 0s 115us/step - loss: 0.2042 - accuracy: 0.9551 - val_loss: 0.8555 - val_accuracy: 0.5974
Epoch 226/1000
178/178 [==============================] - 0s 105us/step - loss: 0.2039 - accuracy: 0.9438 - val_loss: 0.8546 - val_accuracy: 0.5974
Epoch 227/1000
178/178 [==============================] - 0s 103us/step - loss: 0.2024 - accuracy: 0.9494 - val_loss: 0.8559 - val_accuracy: 0.5974
Epoch 228/1000
178/178 [==============================] - 0s 101us/step - loss: 0.2019 - accuracy: 0.9494 - val_loss: 0.8563 - v

...

178/178 [==============================] - 0s 113us/step - loss: 0.0710 - accuracy: 0.9775 - val_loss: 1.7731 - val_accuracy: 0.6623
Epoch 773/1000
178/178 [==============================] - 0s 118us/step - loss: 0.0719 - accuracy: 0.9775 - val_loss: 1.7765 - val_accuracy: 0.6623
Epoch 774/1000
178/178 [==============================] - 0s 109us/step - loss: 0.0698 - accuracy: 0.9775 - val_loss: 1.7808 - val_accuracy: 0.6623
Epoch 775/1000
178/178 [==============================] - 0s 102us/step - loss: 0.0701 - accuracy: 0.9775 - val_loss: 1.7708 - val_accuracy: 0.6623
Epoch 776/1000
178/178 [==============================] - 0s 110us/step - loss: 0.0707 - accuracy: 0.9775 - val_loss: 1.7834 - val_accuracy: 0.6623
Epoch 777/1000
178/178 [==============================] - 0s 105us/step - loss: 0.0697 - accuracy: 0.9775 - val_loss: 1.7921 - val_accuracy: 0.6234
Epoch 778/1000
178/178 [==============================] - 0s 117us/step - loss: 0.0706 - accuracy: 0.9775 - val_loss: 1.8075 - 

178/178 [==============================] - 0s 115us/step - loss: 0.0706 - accuracy: 0.9775 - val_loss: 1.8315 - val_accuracy: 0.6623
Epoch 828/1000
178/178 [==============================] - 0s 125us/step - loss: 0.0681 - accuracy: 0.9775 - val_loss: 1.8301 - val_accuracy: 0.6623
Epoch 829/1000
178/178 [==============================] - 0s 115us/step - loss: 0.0675 - accuracy: 0.9775 - val_loss: 1.8360 - val_accuracy: 0.6623
Epoch 830/1000
178/178 [==============================] - 0s 128us/step - loss: 0.0681 - accuracy: 0.9775 - val_loss: 1.8453 - val_accuracy: 0.6623
Epoch 831/1000
178/178 [==============================] - 0s 130us/step - loss: 0.0687 - accuracy: 0.9775 - val_loss: 1.8364 - val_accuracy: 0.6623
Epoch 832/1000
178/178 [==============================] - 0s 93us/step - loss: 0.0683 - accuracy: 0.9775 - val_loss: 1.8484 - val_accuracy: 0.6753
Epoch 833/1000
178/178 [==============================] - 0s 113us/step - loss: 0.0692 - accuracy: 0.9775 - val_loss: 1.8508 - v

178/178 [==============================] - 0s 114us/step - loss: 0.0655 - accuracy: 0.9831 - val_loss: 1.8938 - val_accuracy: 0.6753
Epoch 883/1000
178/178 [==============================] - 0s 84us/step - loss: 0.0659 - accuracy: 0.9831 - val_loss: 1.9128 - val_accuracy: 0.6623
Epoch 884/1000
178/178 [==============================] - 0s 94us/step - loss: 0.0655 - accuracy: 0.9831 - val_loss: 1.8963 - val_accuracy: 0.6753
Epoch 885/1000
178/178 [==============================] - 0s 121us/step - loss: 0.0659 - accuracy: 0.9775 - val_loss: 1.9115 - val_accuracy: 0.6623
Epoch 886/1000
178/178 [==============================] - 0s 119us/step - loss: 0.0664 - accuracy: 0.9831 - val_loss: 1.9116 - val_accuracy: 0.6623
Epoch 887/1000
178/178 [==============================] - 0s 98us/step - loss: 0.0640 - accuracy: 0.9831 - val_loss: 1.9107 - val_accuracy: 0.6753
Epoch 888/1000
178/178 [==============================] - 0s 105us/step - loss: 0.0654 - accuracy: 0.9775 - val_loss: 1.9154 - val

178/178 [==============================] - 0s 119us/step - loss: 0.0631 - accuracy: 0.9775 - val_loss: 1.9604 - val_accuracy: 0.6753
Epoch 938/1000
178/178 [==============================] - 0s 112us/step - loss: 0.0629 - accuracy: 0.9831 - val_loss: 1.9459 - val_accuracy: 0.6623
Epoch 939/1000
178/178 [==============================] - 0s 131us/step - loss: 0.0644 - accuracy: 0.9831 - val_loss: 1.9597 - val_accuracy: 0.6753
Epoch 940/1000
178/178 [==============================] - 0s 116us/step - loss: 0.0641 - accuracy: 0.9775 - val_loss: 1.9809 - val_accuracy: 0.6364
Epoch 941/1000
178/178 [==============================] - 0s 120us/step - loss: 0.0664 - accuracy: 0.9831 - val_loss: 1.9719 - val_accuracy: 0.6753
Epoch 942/1000
178/178 [==============================] - 0s 104us/step - loss: 0.0655 - accuracy: 0.9775 - val_loss: 1.9762 - val_accuracy: 0.6623
Epoch 943/1000
178/178 [==============================] - 0s 101us/step - loss: 0.0641 - accuracy: 0.9775 - val_loss: 1.9767 - 

178/178 [==============================] - 0s 109us/step - loss: 0.0625 - accuracy: 0.9831 - val_loss: 2.0031 - val_accuracy: 0.6753
Epoch 993/1000
178/178 [==============================] - 0s 120us/step - loss: 0.0616 - accuracy: 0.9831 - val_loss: 2.0200 - val_accuracy: 0.6753
Epoch 994/1000
178/178 [==============================] - 0s 129us/step - loss: 0.0636 - accuracy: 0.9831 - val_loss: 2.0359 - val_accuracy: 0.6364
Epoch 995/1000
178/178 [==============================] - 0s 109us/step - loss: 0.0618 - accuracy: 0.9831 - val_loss: 2.0082 - val_accuracy: 0.6753
Epoch 996/1000
178/178 [==============================] - 0s 106us/step - loss: 0.0601 - accuracy: 0.9831 - val_loss: 2.0333 - val_accuracy: 0.6364
Epoch 997/1000
178/178 [==============================] - 0s 103us/step - loss: 0.0615 - accuracy: 0.9831 - val_loss: 1.9984 - val_accuracy: 0.6753
Epoch 998/1000
178/178 [==============================] - 0s 102us/step - loss: 0.0627 - accuracy: 0.9831 - val_loss: 2.0220 - 

In [260]:
acc_test_sel2 = model_sel2.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel2*100))

77/77 [==============================] - 0s 112us/step
test accuracy: 66.23%


In [261]:
############ Logistic Regression ############

In [262]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [270]:
###### logistics
log = LogisticRegression()
log.fit(X_train, y_train)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [271]:
y_pred = log.predict(X_test)
print('combination logistic test accuracy %.2f%%' % (log.score(X_test, y_test)*100))

combination logistic test accuracy 62.34%


In [265]:
############## Random Forest ##############

In [266]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train,y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [267]:
from sklearn.metrics import accuracy_score
y_pred = rf.predict(X_test)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test, y_pred)*100))

combination test accuracy: 63.64%


In [268]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [269]:
## random forest model with CV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv = RandomForestClassifier(n_estimators=100, random_state=123)

accs = cross_val_score(estimator=rfcv, X=X_train, y=y_train, cv=5)
print(accs)
print(accs.mean())

[0.80555556 0.80555556 0.63888889 0.77777778 0.73529412]
0.7526143790849673
